In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolorsx
from matplotlib.patches import Circle

from regions import Regions

import os
from glob import glob

from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import proj_plane_pixel_scales
from astropy.wcs.utils import pixel_to_skycoord, skycoord_to_pixel
from astropy.nddata import Cutout2D
import astropy.units as u
from astropy.coordinates import SkyCoord

from reproject import reproject_exact, reproject_interp

from AstroColour.AstroColour import RGB

import matplotlib.patheffects as patheffects
import matplotlib.patheffects as PathEffects

%matplotlib widget

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
fig_width_pt = 244.0  # Get this from LaTeX using \the\columnwidth
text_width_pt = 508.0 # Get this from LaTeX using \the\textwidth

inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt*1.5 # width in inches
fig_width_full = text_width_pt*inches_per_pt*1.5  # 17
fig_height =fig_width*golden_mean # height in inches
fig_size = [fig_width,fig_height] #(9,5.5) #(9, 4.5)
fig_height_full = fig_width_full*golden_mean

In [ ]:
path = '/Users/zgl12/Python_Scripts/Image_Processing/MAST/MAST_2025-11-25T20_43_45.354Z/'
reg_file1 = '/Users/zgl12/Downloads/critics_zs5.13_mu100_wcs.reg'
reg_file2 = '/Users/zgl12/Downloads/critics_zs5.13_wcs.reg'
name = f'jw06882-o038_t041_nircam_clear-f115w_i2d.fits'

hdu = fits.open(path+name)
data = hdu[1].data
header = hdu[1].header
hdr = hdu[0].header
wcs = WCS(header)
# hdu.info()
# print(repr(header))
hdu.close()

regions1 = Regions.read(reg_file1)
# regions2 = Regions.read(reg_file2)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)

# Plot the FITS image
ax.imshow(data, origin='lower', cmap='gray', 
          vmin=np.nanpercentile(data, 35),
          vmax=np.nanpercentile(data, 99.5))

# Overlay the DS9 regions
for region in regions1:
    pix_region = region.to_pixel(wcs)
    pix_region.plot(ax=ax, color='red', linewidth=1)

# for region in regions2:
#     pix_region = region.to_pixel(wcs)
#     pix_region.plot(ax=ax, color='red', linewidth=1)

ax.set_xlabel("RA")
ax.set_ylabel("Dec")
plt.show()

In [ ]:
red = np.load('sneos_red.npy')
green = np.load('sneos_green.npy')
blue = np.load('sneos_blue.npy')

In [ ]:
def compute_output_shape(w_old, w_new, ny, nx):
    # Corners of original image
    corners = np.array([
        [0,     0    ],
        [nx-1,  0    ],
        [0,     ny-1 ],
        [nx-1,  ny-1]
    ])

    sky = pixel_to_skycoord(corners[:,0], corners[:,1], w_old)
    x_new, y_new = skycoord_to_pixel(sky, w_new)

    xmin, xmax = np.min(x_new), np.max(x_new)
    ymin, ymax = np.min(y_new), np.max(y_new)

    new_nx = int(np.ceil(xmax - xmin))
    new_ny = int(np.ceil(ymax - ymin))

    shift_x = -xmin
    shift_y = -ymin

    return new_ny, new_nx, shift_x, shift_y

In [ ]:
w = wcs.celestial
ny, nx = data.shape

new_wcs = WCS(naxis=2)
new_wcs.wcs.crval = w.wcs.crval
new_wcs.wcs.crpix = [nx/2, ny/2]
new_wcs.wcs.ctype = ["RA---TAN", "DEC--TAN"]

scale = 8.67346751159964e-06  # arcsec/pixel in degrees
new_wcs.wcs.cdelt = [scale, -scale]   # East-left, North-up

new_wcs.wcs.pc = np.eye(2)

new_ny, new_nx, sx, sy = compute_output_shape(w, new_wcs, ny, nx)
new_wcs.wcs.crpix += [sx, sy]

In [ ]:

center = SkyCoord("19h31m49.5s", "-26d34m30s")   # add your Dec here if known

dra = 40 * u.arcsec        # RA half-width
ddec = 40 * u.arcsec       # Dec half-width

west  = SkyCoord(center.ra  - dra, center.dec)   # smaller RA → west
east  = SkyCoord(center.ra  + dra, center.dec)   # larger RA → east
south = SkyCoord(center.ra, center.dec - ddec)
north = SkyCoord(center.ra, center.dec + ddec)

cx_w, cy_s  = new_wcs.world_to_pixel(west)
cx_e, cy_n  = new_wcs.world_to_pixel(east)
cx_c, cy_c  = new_wcs.world_to_pixel(center)
sx, sy      = new_wcs.world_to_pixel(south)
nx_, ny_    = new_wcs.world_to_pixel(north)

print(f'Center pixel coords: ({cx_c:.1f}, {cy_c:.1f})')
print(f'X pixel bounds:   ({cx_e:.1f}, {cx_w:.1f})')
print(f'Y pixel bounds:   ({sy:.1f}, {ny_:.1f})')

In [ ]:
init_colour_cube = [red, green, blue]
colour_cube = []

for i in range(len(init_colour_cube)):
    temp_data, _ = reproject_interp((init_colour_cube[i], w), new_wcs, shape_out=(new_ny, new_nx))
    colour_cube.append(temp_data)
    
    
    
    plt.figure()
    plt.imshow(temp_data, origin='lower', cmap='gray', vmin=np.nanpercentile(temp_data, 1), vmax=np.nanpercentile(temp_data, 99))
    plt.show()

In [ ]:
rgb = RGB(colour_cube,
          save = False, save_name = 'snEos', save_folder = '/Users/zgl12/', 
          epsf_plot=False, epsf = False,
          bkg_plot = False, temp_save = True, run = False, manual_override=0)

In [ ]:
colour = rgb.master_plot(colour_cube, 
                         colours = ['red', 'green', 'blue'],
                         intensities = [0.6, 1, 0.56], 
                         gamma = [0.95, 0.95, 0.95],
                         norms = ['asinh', 'asinh', 'asinh'], 
                         uppers = [99, 99, 99],
                         lowers = [5, 5, 5], 
                         interactive=False)

In [ ]:
np.save('/Users/zgl12/sneos_rgb.npy', colour)

In [ ]:

center = SkyCoord("19h31m49.5s", "-26d34m30s")   # add your Dec here if known

dra = 25 * u.arcsec        # RA half-width
ddec = 25 * u.arcsec       # Dec half-width

west  = SkyCoord(center.ra  - dra, center.dec)   # smaller RA → west
east  = SkyCoord(center.ra  + dra, center.dec)   # larger RA → east
south = SkyCoord(center.ra, center.dec - ddec)
north = SkyCoord(center.ra, center.dec + ddec)

cx_w, cy_s  = new_wcs.world_to_pixel(west)
cx_e, cy_n  = new_wcs.world_to_pixel(east)
cx_c, cy_c  = new_wcs.world_to_pixel(center)
sx, sy      = new_wcs.world_to_pixel(south)
nx_, ny_    = new_wcs.world_to_pixel(north)

print(f'Center pixel coords: ({cx_c:.1f}, {cy_c:.1f})')
print(f'X pixel bounds:   ({cx_e:.1f}, {cx_w:.1f})')
print(f'Y pixel bounds:   ({sy:.1f}, {ny_:.1f})')

In [ ]:
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
from matplotlib.patches import Circle, Rectangle

In [ ]:
center = SkyCoord("19h31m49.5s", "-26d34m30s")   # add your Dec here if known

dra = 25 * u.arcsec        # RA half-width
ddec = 25 * u.arcsec       # Dec half-width

west  = SkyCoord(center.ra  - dra, center.dec)   # smaller RA → west
east  = SkyCoord(center.ra  + dra, center.dec)   # larger RA → east
south = SkyCoord(center.ra, center.dec - ddec)
north = SkyCoord(center.ra, center.dec + ddec)

cx_w, cy_s  = new_wcs.world_to_pixel(west)
cx_e, cy_n  = new_wcs.world_to_pixel(east)
cx_c, cy_c  = new_wcs.world_to_pixel(center)
sx, sy      = new_wcs.world_to_pixel(south)
nx_, ny_    = new_wcs.world_to_pixel(north)

fig = plt.figure(figsize=(fig_width_full/1.2, fig_width_full/1.2))
axs = fig.add_subplot(1,1,1, projection=new_wcs)
axs.imshow(colour)   # usual orientation
axs.invert_xaxis()
axs.invert_yaxis()

axs.set_xlim(cx_e, cx_w)   # left = East (larger RA), right = West (smaller RA)
axs.set_ylim(sy, ny_)      # y increases upward (Dec increasing)


dRA = (10 * u.arcsec)  # East arrow length
dDec = (10 * u.arcsec) # North arrow length

end_points = SkyCoord(center.ra - dra, center.dec - ddec, frame='icrs')

arrow_east  = SkyCoord(center.ra + dRA - dra, center.dec - ddec, frame='icrs')
arrow_north = SkyCoord(center.ra - dra, center.dec + dDec - ddec, frame='icrs')

cnx, cny     = new_wcs.world_to_pixel(end_points)
ex, ey     = new_wcs.world_to_pixel(arrow_east)
nx_, ny_   = new_wcs.world_to_pixel(arrow_north)

axs.annotate('', xy=(ex+40, ey-40), xytext=(cnx+40, cny-40),
            arrowprops=dict(arrowstyle='->', lw=2.5, color='white'))
axs.text(ex+70, ey - 32, r'\textbf{E}', color='white', fontsize = 15)

axs.annotate('', xy=(nx_+40, ny_-40), xytext=(cnx+40, cny-40),
            arrowprops=dict(arrowstyle='->', lw=2.5, color='white'))
axs.text(nx_ + 55, ny_ - 45, r'\textbf{N}', color='white', fontsize=15)

axs.text(ex+40, ey - 70, r"$10.0''.=46.6\;{\rm kpc}$", color='white', fontsize = 15)

axs.text(ex+300, ey - 1550, r"JWST/NIRCam -- Sep 2025", color='white', fontsize = 15)

# merged_regions = Regions(list(regions1) + list(regions2))

for region in regions1:
    pix_region = region.to_pixel(new_wcs)
    pix_region.plot(ax=axs, color='limegreen', linewidth=1, alpha = 0.4)
    
# for region in regions2:
#     pix_region = region.to_pixel(new_wcs)
#     pix_region.plot(ax=axs, color='limegreen', linewidth=1, alpha = 0.4)

start_x = ex + 450
start_y = ey - 1520

pieces = [
    (r"\textbf{F115W$+$F150W}", "lightskyblue", 0),
    (r" \textbf{/} ", "white", -230),
    (r"\textbf{F200W$+$F277W}", "limegreen", -245),
    (r" \textbf{/} ", "white", -475),
    (r"\textbf{F356W$+$F444W}", "red", -490),
]

# fig.canvas.draw()
# renderer = fig.canvas.get_renderer()

for txt, color, edge in pieces:
    
    t = axs.text(start_x + edge, start_y, txt, color=color, va="top", ha="left", 
                    transform=axs.transData, clip_on=True, fontsize = 12, zorder=1000)

    t.set_path_effects([PathEffects.withStroke(linewidth=1.3, foreground="black", 
                                            alpha=0.5), PathEffects.Normal()])

fig.canvas.draw()  # ensure layout is known

axins = fig.add_axes([0.07, 0.58, 0.4, 0.4])

x0, y0 = 3885, 8078
zoom_half = 40
axins.set_xlim(x0 + zoom_half, x0 - zoom_half)
axins.set_ylim(y0 + zoom_half, y0 - zoom_half)

axins.imshow(colour, origin='lower')

for region in regions1:
    pix_region = region.to_pixel(new_wcs)
    pix_region.plot(ax=axins, color='limegreen', linewidth=1, alpha = 0.4)
    

for s in axins.spines.values():
    s.set_edgecolor("white")
    s.set_linewidth(2)

axins.set_xticks([])
axins.set_yticks([])

axins.text(x0 - 4, y0 + zoom_half - 10, r"\textbf{SN Eos}", 
           color="white", ha="center", va="top", fontsize = 24, clip_on=True)

targets = [(3877.2, 8063.6), (3893.5, 8089.4)]
radius = 7

iii = [r'101.2', r'101.1']
ii = 0 

for (tx, ty) in targets:
    c = Circle((tx, ty), radius, edgecolor="white", facecolor="none", lw=1.6)
    axins.add_patch(c)
    
    axins.text(tx + 19, ty - 2, iii[ii], color="white", ha="left", va="top", fontsize=15, clip_on=True)
    
    ii += 1
    
mark_inset(axs, axins, loc1=1, loc2=3, fc="none", ec="white", lw=2)

fig.canvas.draw()

axs.coords[0].set_ticks_visible(False)
axs.coords[1].set_ticks_visible(False)

axs.coords[0].set_ticklabel_visible(False)
axs.coords[1].set_ticklabel_visible(False)

axs.coords[0].set_axislabel('')
axs.coords[1].set_axislabel('')

axs.set_frame_on(False)
axs.patch.set_visible(False)

# Remove all figure–axes margins
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

# Make sure text/arrows/inset included
fig.tight_layout(pad=0)

plt.savefig("sneos_recolour_final.pdf",
            bbox_inches="tight",
            pad_inches=0)
plt.show()